### Tools
Models can request to call tools that perform tasks such as fetching data from a database, searching the web, or running code. Tools are pairings of:
1. A schema, including the name of the tool, a description, and/or argument definitions (often a JSON schema)
2. A function or co-routine to execute.

In [3]:
import os
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
load_dotenv()

os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')

model = init_chat_model("groq:qwen/qwen3-32b")
response = model.invoke('Any current big new about AI?')
response

AIMessage(content="<think>\nOkay, the user is asking about the latest major news in AI. Let me start by recalling what I know. There have been some big developments in 2023-2024. First, I should think about the major companies leading in AI. OpenAI, Google, Meta, and Microsoft come to mind. \n\nOpenAI has been working on GPT-4 and might have a new version upcoming. Then there's the recent launch of GPT-4o, which is an open-source version. Wait, was that actually released? I need to verify that. Maybe it's part of their public release, but I should check if that's accurate.\n\nGoogle has been pushing their Gemini models. The latest version, Gemini 1.5, has an extended context window. That's significant because handling longer contexts is a big deal for tasks like document analysis or conversations. They might have also introduced Gemini Nano for on-device processing, which is cool for privacy and efficiency.\n\nMeta's Llama 3 series is another key player. They've been updating their mod

In [5]:
from langchain.tools import tool

@tool
def get_weather(location: str) ->str:
    """Get the weather at a location..."""
    return f"It's sunny in {location}"

model_with_tools = model.bind_tools([get_weather])
model_with_tools

RunnableBinding(bound=ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 16384, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x00000257B985C1A0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000257B985CC20>, model_name='qwen/qwen3-32b', model_kwargs={}, groq_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'get_weather', 'description': 'Get the weather at a location...', 'parameters': {'properties': {'location': {'type': 'string'}}, 'required': ['location'], 'type': 'object'}}}]}, config={}, config_factories=[])

In [7]:
response = model_with_tools.invoke("What's the weather like in Patna?")
print(response)

content='' additional_kwargs={'reasoning_content': 'Okay, the user is asking about the weather in Patna. I need to use the get_weather function for that. The function requires the location parameter. Patna is a city in India, so I should specify that as the location. Let me make sure the parameter is correctly formatted. I\'ll call the function with location set to "Patna".\n', 'tool_calls': [{'id': 'shm9rz8n2', 'function': {'arguments': '{"location":"Patna"}', 'name': 'get_weather'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 96, 'prompt_tokens': 155, 'total_tokens': 251, 'completion_time': 0.161741138, 'completion_tokens_details': {'reasoning_tokens': 71}, 'prompt_time': 0.006356361, 'prompt_tokens_details': None, 'queue_time': 0.062880449, 'total_time': 0.168097499}, 'model_name': 'qwen/qwen3-32b', 'system_fingerprint': 'fp_5cf921caa2', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'} id='lc_ru

In [8]:
for tool_call in response.tool_calls:
    print(f"Tool: {tool_call['name']}")
    print(f"Args: {tool_call['args']}")

Tool: get_weather
Args: {'location': 'Patna'}


### Tool execution loops

In [10]:
## step 1 : model generates tool calls

messages = [{
    'role':'user',
    'content': 'What is the weather in Patna?'
}]

ai_message = model_with_tools.invoke(messages)
messages.append(ai_message)

messages

[{'role': 'user', 'content': 'What is the weather in Patna?'},
 AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, the user is asking for the weather in Patna. I need to use the get_weather function. The function requires a location parameter. Patna is a city in India, so I should pass "Patna" as the location. Let me check if there are any other parameters needed, but the tool only asks for location. Alright, I\'ll format the tool call with the name as get_weather and arguments as {"location": "Patna"}.\n', 'tool_calls': [{'id': 'rh02c5ddd', 'function': {'arguments': '{"location":"Patna"}', 'name': 'get_weather'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 118, 'prompt_tokens': 154, 'total_tokens': 272, 'completion_time': 0.178541277, 'completion_tokens_details': {'reasoning_tokens': 93}, 'prompt_time': 0.00627235, 'prompt_tokens_details': None, 'queue_time': 0.159132154, 'total_time': 0.184813627}, 'model_name': 'qwen/qwen3-32b', '

In [11]:
## step 2 : execute tool and collect results

for tool_call in ai_message.tool_calls:
    tool_result = get_weather.invoke(tool_call)
    messages.append(tool_result)

messages

[{'role': 'user', 'content': 'What is the weather in Patna?'},
 AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, the user is asking for the weather in Patna. I need to use the get_weather function. The function requires a location parameter. Patna is a city in India, so I should pass "Patna" as the location. Let me check if there are any other parameters needed, but the tool only asks for location. Alright, I\'ll format the tool call with the name as get_weather and arguments as {"location": "Patna"}.\n', 'tool_calls': [{'id': 'rh02c5ddd', 'function': {'arguments': '{"location":"Patna"}', 'name': 'get_weather'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 118, 'prompt_tokens': 154, 'total_tokens': 272, 'completion_time': 0.178541277, 'completion_tokens_details': {'reasoning_tokens': 93}, 'prompt_time': 0.00627235, 'prompt_tokens_details': None, 'queue_time': 0.159132154, 'total_time': 0.184813627}, 'model_name': 'qwen/qwen3-32b', '

In [12]:
## step 3 : Pass results back to model for final response

final_response = model_with_tools.invoke(messages)
final_response

AIMessage(content="The current weather in Patna is sunny. Let me know if you'd like to check another location! ☀️", additional_kwargs={'reasoning_content': "Okay, the user asked about the weather in Patna. I called the get_weather function with Patna as the location. The response came back saying it's sunny there. Now I need to relay that information clearly. Let me check if there's any additional details I should mention, but the response seems straightforward. Just state the current weather condition in a friendly manner. Make sure to confirm the location and the weather status. No need for extra fluff. Keep it concise and accurate.\n"}, response_metadata={'token_usage': {'completion_tokens': 127, 'prompt_tokens': 194, 'total_tokens': 321, 'completion_time': 0.239626797, 'completion_tokens_details': {'reasoning_tokens': 98}, 'prompt_time': 0.008230082, 'prompt_tokens_details': None, 'queue_time': 0.056398982, 'total_time': 0.247856879}, 'model_name': 'qwen/qwen3-32b', 'system_fingerp